In [2]:
import pandas as pd

df = pd.read_csv('smote.csv',decimal=',')
df = df.drop(columns='Unnamed: 0')
df.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,is_anomaly
0,Italy,150.00,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
1,Greece,3200.00,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
2,Italy,8350.00,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
3,Italy,500.00,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL,0
4,Italy,1895.68,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL,0


In [84]:
#df=pd.read_csv('misc_anomalies.csv',decimal=',')
#df.head()

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181862 entries, 0 to 181861
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   country                  181862 non-null  object 
 1   amount                   181862 non-null  float64
 2   userAgent_isPc           181862 non-null  int64  
 3   userAgent_browserFamily  181862 non-null  object 
 4   userAgent_osFamily       181862 non-null  object 
 5   IBAN_countryCode         181862 non-null  object 
 6   IBAN_bankCode            181862 non-null  object 
 7   SIA                      181862 non-null  object 
 8   CAP                      181862 non-null  int64  
 9   SAE                      181862 non-null  int64  
 10  RAE                      181862 non-null  int64  
 11  companyType              181862 non-null  object 
 12  is_anomaly               181862 non-null  int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 18.0+ MB


In [3]:
df['CAP'] = df['CAP'].apply(str)
df['SAE'] = df['SAE'].apply(str)
df['RAE'] = df['RAE'].apply(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181862 entries, 0 to 181861
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   country                  181862 non-null  object 
 1   amount                   181862 non-null  float64
 2   userAgent_isPc           181862 non-null  int64  
 3   userAgent_browserFamily  181862 non-null  object 
 4   userAgent_osFamily       181862 non-null  object 
 5   IBAN_countryCode         181862 non-null  object 
 6   IBAN_bankCode            181862 non-null  object 
 7   SIA                      181862 non-null  object 
 8   CAP                      181862 non-null  object 
 9   SAE                      181862 non-null  object 
 10  RAE                      181862 non-null  object 
 11  companyType              181862 non-null  object 
 12  is_anomaly               181862 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 18.0+ MB


In [4]:
import pickle
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))

In [5]:
columns_to_label = ['country','userAgent_browserFamily','userAgent_osFamily','IBAN_countryCode','IBAN_bankCode','CAP','SIA','SAE','RAE','companyType']
for clmn in columns_to_label:
    df[clmn]=df[clmn].apply(str)
    df[clmn] = encoder_dict[clmn].transform(df[clmn])
df.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,is_anomaly
0,6,150.00,1,4,5,17,699,2013,141,19,113,5,0
1,5,3200.00,1,4,5,17,699,2013,141,19,113,5,0
2,6,8350.00,1,4,5,17,699,2013,141,19,113,5,0
3,6,500.00,1,0,5,17,123,1781,87,8,99,10,0
4,6,1895.68,1,0,5,17,285,1781,87,8,99,10,0


In [41]:
X=df.sample(frac=1,random_state=0)
y=X['is_anomaly']
X=X.drop(columns='is_anomaly')

In [89]:
#PER TESTARE MISC
#df=df.drop(columns=['timestamp','businessName','customerId'])

In [6]:
X_train=df.sample(frac=0.8, random_state=0)
#X_train=df.sample(frac=0.5, random_state=0)
y_train=X_train['is_anomaly']
X_train=X_train.drop(columns='is_anomaly')

X_test=df.drop(X_train.index)
y_test=X_test['is_anomaly']
X_test=X_test.drop(columns='is_anomaly')

In [91]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
neigh.predict(X_test)
#neigh.predict(X_test.loc[y_test[y_test==1].index]              )

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [8]:
prediction=neigh.predict(X_test)

In [9]:
import numpy as np
np.array_equal(prediction,y_test.to_numpy)

False

In [10]:
print(prediction==y_test.to_numpy)

[False False False ... False False False]


In [11]:
y_test.to_numpy

<bound method IndexOpsMixin.to_numpy of 2         0
10        0
21        0
27        0
30        0
         ..
181836    1
181840    1
181842    1
181850    1
181860    1
Name: is_anomaly, Length: 36372, dtype: int64>

In [60]:
y_test

2        0
4        0
7        0
10       0
13       0
        ..
90950    0
90951    0
90952    0
90956    0
90957    0
Name: is_anomaly, Length: 45480, dtype: int64

In [63]:
y_test.unique()

array([0, 1], dtype=int64)

In [65]:
y_test['is_anomaly']

KeyError: 'is_anomaly'

In [92]:
neigh.score(X_test,y_test)

0.9927141757395799

In [21]:
y_np=y_test.to_numpy()

In [22]:
type(y_np)

numpy.ndarray

In [23]:
z=0
for x,y in zip(prediction,y_np):
    if x == y:
        z=z+1
print(z)


36107


In [25]:
z=0
for x in y_np:
    if x==1:
        z=z+1
print(z)


18086


In [27]:
misc_test=pd.read_csv('misc_test.csv',decimal=',')
misc_test.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,is_anomaly
0,Austria,264986.74,1,Chrome,Windows,AT,34000,BUV4U,39031,430,161,SRL,1
1,Italy,111345.77,1,Chrome,Windows,DE,34270024,B5SDZ,39100,615,11,SS,0
2,Italy,1659.00,1,Firefox,Windows,IT,3069,BQ6RB,31010,430,649,SRL,0
3,Italy,200.00,1,Safari,Mac OS X,IT,8794,CM5CR,30174,492,614,SAS,1
4,Italy,1000.00,1,Chrome,Windows,IT,8327,CEFGV,31039,430,505,SRL,0


In [28]:
misc_test['CAP'] = misc_test['CAP'].apply(str)
misc_test['SAE'] = misc_test['SAE'].apply(str)
misc_test['RAE'] = misc_test['RAE'].apply(str)
misc_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  9 non-null      object 
 1   amount                   9 non-null      float64
 2   userAgent_isPc           9 non-null      int64  
 3   userAgent_browserFamily  9 non-null      object 
 4   userAgent_osFamily       9 non-null      object 
 5   IBAN_countryCode         9 non-null      object 
 6   IBAN_bankCode            9 non-null      int64  
 7   SIA                      9 non-null      object 
 8   CAP                      9 non-null      object 
 9   SAE                      9 non-null      object 
 10  RAE                      9 non-null      object 
 11  companyType              9 non-null      object 
 12  is_anomaly               9 non-null      int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 1.0+ KB


In [29]:
for clmn in columns_to_label:
    misc_test[clmn]=misc_test[clmn].apply(str)
    misc_test[clmn] = encoder_dict[clmn].transform(misc_test[clmn])

misc_test.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,is_anomaly
0,0,264986.74,1,0,5,0,364,1482,302,8,4,10,1
1,6,111345.77,1,0,5,6,376,965,327,25,1,11,0
2,6,1659.00,1,4,5,17,285,1346,62,8,121,10,0
3,6,200.00,1,11,3,17,1217,2045,58,19,106,5,1
4,6,1000.00,1,0,5,17,1088,1781,87,8,99,10,0


In [31]:
neigh.predict(misc_test.drop(columns='is_anomaly'))

array([1, 1, 0, 1, 0, 1, 1, 0, 0], dtype=int64)

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,is_anomaly
0,93000801,2021-09-15 16:13:35.147,Italy,150.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
1,93000801,2021-08-24 10:34:24.487,Greece,3200.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
2,93000801,2021-09-14 16:21:43.33,Italy,8350.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
3,93001001,2021-06-30 09:07:16.733,Italy,500.00,3f241dfcc38064261fec0f303c2784e5,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL,0
4,93001001,2021-07-12 16:36:02.857,Italy,1895.68,a5d1eaffa8e325d370665f43624339a1,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL,0


In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score

for k in range(2,20):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train)
    #print(k)
    #y_score=neigh.predict_proba(X_test)[:,1]
    y_pred=neigh.predict(X_test)
    #print(f1_score(y_test,y_pred)) #diminuisce
    #print(brier_score_loss(y_test,y_score)) #lower better aumenta
    #print(average_precision_score(y_test,y_score,average='micro')) #aumenta
    #print(roc_auc_score(y_test,y_score)) #aumenta
    #print(neigh.score(X_test,y_test)) #diminuisce

0.9930912507599624
0.9927071579932301
0.9925856508916513
0.9918046312084045
0.991351291797499
0.9908690869086908
0.9905291558834867
0.9902944653707625
0.9904227212681638
0.9895870538780669
0.9894922154370908
0.9890375580404979
0.9889729135157431
0.9885171144442614
0.9884526558891455
0.987915407854985
0.987682832948422
0.9873389909642691


In [37]:
neigh.predict(X_test)

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [14]:
neigh.predict_proba(X_test)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,neigh.predict_proba(X_test)[:,1])

0.9941180144452116

In [23]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)
#print(k)
score=roc_auc_score(neigh.predict_proba(X_test)[:,1],y_test)
print(score)

KNeighborsClassifier(n_neighbors=10)

In [28]:
neigh.predict_proba(X_test)[:,1]

array([0.1, 0. , 0. , ..., 1. , 1. , 1. ])

In [42]:
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()

k_range = list(range(1,100))
weight_options = ["uniform", "distance"]

param_grid = dict(n_neighbors = k_range, weights = weight_options)

grid = GridSearchCV(knn, param_grid, cv = 10, scoring = 'accuracy')
grid.fit(X,y)

print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

0.9943143672710487
{'n_neighbors': 1, 'weights': 'uniform'}
KNeighborsClassifier(n_neighbors=1)
